<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [43]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [44]:
df = df.sample(False, 0.3)

In [45]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [46]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [47]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [13]:
index_columns = [i+'_index' for i in cat_columns]
string_indexers = [StringIndexer(inputCol=cat, outputCol=ind, handleInvalid='keep') 
                   for cat, ind in zip(cat_columns, index_columns)]

In [14]:
ohe_columns = [i+'_ohe' for i in cat_columns]
ohe_estimator = OneHotEncoderEstimator(inputCols=index_columns, outputCols=ohe_columns)

In [15]:
vector_assembler = VectorAssembler(inputCols=num_columns+ohe_columns, outputCol='features')

In [16]:
pipeline = Pipeline(stages=[*string_indexers,
                                            ohe_estimator,
                                            vector_assembler])

### Fit and save pipeline

In [17]:
model = pipeline.fit(df)

In [18]:
model.write().overwrite().save(DATA_PATH+'/pipeline_model')

### Load fitted pipeline

In [19]:
from pyspark.ml import PipelineModel


In [20]:
loaded_pipeline_model = PipelineModel.load(DATA_PATH+'/pipeline_model')

### Transform dataset using pipeline

In [48]:
df = loaded_pipeline_model.transform(df)

In [49]:
df = df.select('id', 'features', F.col('_c0').alias('label'))

In [50]:
df.take(5)

[Row(id=39, features=SparseVector(1949, {0: 8.0, 2: 15.0, 3: 20.0, 4: 115.0, 5: 24.0, 6: 8.0, 7: 23.0, 8: 24.0, 9: 2.0, 10: 2.0, 12: 20.0, 15: 1.0, 1509: 1.0}), label=0),
 Row(id=139, features=SparseVector(1949, {1: 7.0, 2: 1.0, 4: 9128.0, 8: 7.0, 22: 1.0, 1402: 1.0}), label=0),
 Row(id=175, features=SparseVector(1949, {1: 18.0, 2: 15.0, 3: 9.0, 4: 4494.0, 7: 9.0, 8: 8.0, 12: 9.0, 13: 1.0, 1414: 1.0}), label=0),
 Row(id=206, features=SparseVector(1949, {1: 76.0, 2: 3.0, 4: 5029.0, 7: 16.0, 13: 1.0, 1409: 1.0}), label=0),
 Row(id=212, features=SparseVector(1949, {0: 1.0, 1: 5.0, 2: 33.0, 3: 6.0, 4: 375.0, 5: 74.0, 6: 3.0, 7: 46.0, 8: 258.0, 9: 1.0, 10: 2.0, 12: 68.0, 13: 1.0, 1478: 1.0}), label=1)]

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [24]:
from pyspark.sql.window import Window

In [51]:
def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    parts = []
    
    window = Window().orderBy(split_col)
    temp_df = df.withColumn('percent_rank', F.percent_rank().over(window))
    
    min_, max_ = -0.1, 0
    for i in range(len(parts_fractions)):
        max_ += parts_fractions[i]
        part = temp_df.filter((F.col('percent_rank')>min_)&(F.col('percent_rank')<=max_)).drop('percent_rank')
        parts.append(part)
        min_ = max_
    
    return parts

In [52]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [53]:
N = df.count()

In [54]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999994540923512, 0.1000002729538244, 0.1000002729538244)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [55]:
from pyspark.ml.classification import LogisticRegression


lr = LogisticRegression()
lr_model = lr.fit(train_df)

In [56]:
lr_model.transform(val_df).select('probability', 'prediction').take(5)

[Row(probability=DenseVector([0.8249, 0.1751]), prediction=0.0),
 Row(probability=DenseVector([0.6988, 0.3012]), prediction=0.0),
 Row(probability=DenseVector([0.6072, 0.3928]), prediction=0.0),
 Row(probability=DenseVector([0.8048, 0.1952]), prediction=0.0),
 Row(probability=DenseVector([0.4965, 0.5035]), prediction=1.0)]

In [57]:
lr_model.transform(val_df).rdd.take(5)

[Row(id=455266579911, features=SparseVector(1949, {2: 1.0, 4: 826.0, 5: 376.0, 6: 12.0, 8: 266.0, 10: 5.0, 14: 1.0, 1419: 1.0}), label=0, rawPrediction=DenseVector([1.5501, -1.5501]), probability=DenseVector([0.8249, 0.1751]), prediction=0.0),
 Row(id=455266579928, features=SparseVector(1949, {1: 1.0, 4: 21824.0, 5: 152.0, 6: 2.0, 8: 56.0, 10: 2.0, 1232: 1.0, 1402: 1.0}), label=0, rawPrediction=DenseVector([0.8418, -0.8418]), probability=DenseVector([0.6988, 0.3012]), prediction=0.0),
 Row(id=455266579947, features=SparseVector(1949, {0: 11.0, 2: 2.0, 4: 931.0, 5: 40.0, 6: 11.0, 7: 14.0, 8: 14.0, 9: 1.0, 10: 1.0, 13: 1.0, 1449: 1.0}), label=0, rawPrediction=DenseVector([0.4356, -0.4356]), probability=DenseVector([0.6072, 0.3928]), prediction=0.0),
 Row(id=455266580019, features=SparseVector(1949, {2: 7.0, 3: 6.0, 4: 56436.0, 5: 411.0, 7: 6.0, 8: 28.0, 12: 6.0, 17: 1.0, 1402: 1.0}), label=0, rawPrediction=DenseVector([1.4163, -1.4163]), probability=DenseVector([0.8048, 0.1952]), predict

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [32]:
lr_model.transform(val_df) \
    .rdd.map(lambda arr: (arr['probability'][1], arr['label'])) \
    .take(5)

[(0.47825004921833303, 1),
 (0.237196380388703, 0),
 (0.3174236032369763, 1),
 (0.30211353912235545, 0),
 (0.10465153254747395, 0)]

In [59]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics

eps = 0.01

def predict(model, df):
    return lr_model.transform(val_df) \
                    .rdd.map(lambda arr: (float(arr['probability'][1]), float(arr['label'])))

def rocauc(model, df):
    prediction = predict(model, df)
    return BinaryClassificationMetrics(prediction).areaUnderROC

def logloss_func(arr):
    pred, label = arr
    return -(label*np.log(np.clip(pred, eps, 1.-eps)) + 
                             (1.-label)*np.log(1.-np.clip(pred, eps, 1-eps)))

def logloss(model, df):
    prediction = predict(model, df)
    return prediction.map(logloss_func).mean()

def ne(model, df):
    prediction = predict(model, df)
    label_mean = prediction.map(lambda arr: arr[1]).mean()
    
    extremum = logloss_func((label_mean, label_mean))
    return logloss(model, df)/extremum

In [60]:
logloss(lr_model, val_df)

0.5281230113505179

In [61]:
rocauc(lr_model, val_df)

0.7005781346598663

In [62]:
ne(lr_model, val_df)

0.9204193649214951

In [63]:
rocauc(lr_model, test_df)

0.7005781346598663

In [64]:
ne(lr_model, test_df)

0.9204193649214951

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [65]:
import pandas as pd

In [66]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

test_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

test_df = test_df.fillna(0, subset=num_columns)
loaded_pipeline_model = PipelineModel.load(DATA_PATH+'/pipeline_model')
test_df = loaded_pipeline_model.transform(test_df)
test_df = test_df.select('id', 'features')

In [67]:
pred = lr_model.transform(test_df).select('id','probability')
pred = pred.toPandas()
pred['proba'] = pred['probability'].map(lambda arr: arr[1])

In [69]:
pred[['id', 'proba']].to_csv('lr_pred.csv', index=False)